In [1]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=6, suppress=True)

In [2]:
df_train = pd.read_csv('./data/second-order/Centar_train', index_col=0)
df_valid = pd.read_csv('./data/second-order/Centar_validation', index_col=0)
df_test = pd.read_csv('./data/second-order/Centar_test', index_col=0)

In [17]:
df_train.head(10)

,PM10,PM25,hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos,weekend,holiday,...,precip_intensity,precip_probability,temperature,uv_index,visibility,wind_speed,wind_bearing_sin,wind_bearing_cos,PM10_missing,PM25_missing
Timestamp,,,,,,,,,,,,,,,,,,,,,
2011-09-13 18:00:00,-0.330611,0.856226,0.000000,0.500000,0.900969,0.801938,0.900969,0.801938,0.0,0.0,...,0.00000,0.0000,2.009019,0.0,0.6206,0.225955,0.620961,0.985148,0.0,0.0
2011-09-13 19:00:00,-0.408947,0.890166,0.017037,0.629410,0.900969,0.801938,0.900969,0.801938,0.0,0.0,...,0.00000,0.0000,1.804491,0.0,0.6206,0.264156,0.569587,0.995134,0.0,0.0
2011-09-13 20:00:00,-0.468705,0.821216,0.066987,0.750000,0.900969,0.801938,0.900969,0.801938,0.0,0.0,...,0.00000,0.0000,1.521727,0.0,0.6206,0.218640,0.719186,0.949397,0.0,0.0
2011-09-13 21:00:00,0.390181,1.594060,0.146447,0.853553,0.900969,0.801938,0.900969,0.801938,0.0,0.0,...,0.00000,0.0000,1.337316,0.0,0.6206,0.091574,0.328990,0.969846,0.0,0.0
2011-09-13 22:00:00,0.616925,1.795114,0.250000,0.933013,0.900969,0.801938,0.900969,0.801938,0.0,0.0,...,0.00000,0.0000,1.093670,0.0,0.6206,0.087781,0.430413,0.995134,0.0,0.0
2011-09-13 23:00:00,0.024063,1.240560,0.370590,0.982963,0.900969,0.801938,0.900969,0.801938,0.0,0.0,...,0.00000,0.0000,0.878524,0.0,0.6206,0.084801,0.500000,1.000000,0.0,0.0
2011-09-14 00:00:00,-0.052424,1.177626,0.500000,1.000000,1.000000,0.356896,0.900969,0.801938,0.0,0.0,...,0.00000,0.0000,0.815936,0.0,0.6206,0.062043,0.152671,0.859670,0.0,0.0
2011-09-14 01:00:00,-0.216700,1.052516,0.629410,0.982963,1.000000,0.356896,0.900969,0.801938,0.0,0.0,...,0.00000,0.0000,0.701937,0.0,0.6206,0.038472,0.960252,0.695366,0.0,0.0
2011-09-14 02:00:00,-0.436196,0.846731,0.750000,0.933013,1.000000,0.356896,0.900969,0.801938,0.0,0.0,...,0.00000,0.0000,0.615878,0.0,0.6206,0.023300,0.877355,0.171970,0.0,0.0


In [4]:
def build_recurrent_datasets(dataset, mode, history = 24, target_size = 12):
    look_back = history - 1
    start_index = look_back
    end_index = len(dataset) - target_size
    X = dataset.values.copy()
    y = dataset[['PM10', 'PM10_missing']].values.copy()

    data = []
    labels = []
    
    for i in range(start_index, end_index):
        sample_x = X[i-look_back:i+1]
        sample_y = y[i+1:i+1+target_size]
        
        # If we are building a test/validation set and any of the 
        # target values has been imputed, then we discard the sample
        if mode == 'evaluating' and np.any(sample_y[:, 1] == 1):
            continue
            
        data.append(sample_x)
        labels.append(sample_y)

    data = np.array(data).reshape(-1, history, X.shape[1])
    labels = np.array(labels).reshape(-1, target_size, y.shape[1])
    
    assert data.shape[0] == labels.shape[0]

    return data, labels

In [5]:
X_train, y_train = build_recurrent_datasets(df_train, mode='training')
X_valid, y_valid = build_recurrent_datasets(df_valid, mode='evaluating')
X_test, y_test = build_recurrent_datasets(df_test, mode='evaluating')

In [6]:
X_valid.shape

(3389, 24, 23)

In [7]:
np.save('./data/third-order/X_train.npy', X_train)
np.save('./data/third-order/y_train.npy', y_train)
np.save('./data/third-order/X_valid.npy', X_valid)
np.save('./data/third-order/y_valid.npy', y_valid)
np.save('./data/third-order/X_test.npy', X_test)
np.save('./data/third-order/y_test.npy', y_test)